Create surrogate neural population responses during natural behavior trials using variations on dynamical adapting neural models of Baker et al data.

In [ ]:
%matplotlib inline
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy import signal, stats
from sklearn import linear_model
import sys
import warnings; warnings.filterwarnings("ignore")

from aux import get_seg
from disp import set_font_size
from my_stats import nanpearsonr

cc = np.concatenate

FPS = 30.03  # sampling rate of behavioral data
DT = 1/FPS

Load behavioral data from Coen et al 2014

In [ ]:
df_behav = pd.read_csv('data/simple/c_song_f_behav.csv')
df_behav

In [ ]:
# split big df into dfs for individual trials
n_tr = np.max(df_behav.ID) + 1
dfs_tr = [df_behav[df_behav.ID == i] for i in range(n_tr)]

tr_lens = np.array([len(df_tr) for df_tr in dfs_tr])
tr_lens_cum = cc([[0], np.cumsum(tr_lens)])

n_t_total = np.sum(tr_lens)
assert n_t_total == tr_lens_cum[-1]
df_behav = None

# Simulation function

In [ ]:
# define smln & obj function
def smlt(i_s, i_p, tau_r, tau_a, x_s, x_p):
    """Simulate response to song inputs."""
    t = np.arange(len(i_s))*DT
    r = np.nan*np.zeros(len(t))
    
    r[0] = 0
    a_s, a_p = 0, 0
    
    for ct, t_ in enumerate(t[1:], 1):
        a_s += ((DT/tau_a) * (-a_s + x_s*i_s[ct]))
        a_p += ((DT/tau_a) * (-a_p + x_p*i_p[ct]))
        r[ct] = r[ct-1] + (DT/tau_r) * (-r[ct-1] + (x_s - a_s)*i_s[ct] + (x_p - a_p)*i_p[ct])
    
    return r

# Baseline dynamical model params

In [ ]:
# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param

# Dynamical model without adaptation

In [ ]:
SAVE_DIR_DYN = 'data/simple/mlv_c/perturbed_ppln/c_baker_dyn_nonadapt'
SAVE_PFX_DYN = 'mlv_c_baker_dyn_nonadapt'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['TAU_A'] = np.inf
df_dyn_param

In [ ]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

# Dynamical model with homogeneous timescales
(Response and adaptation timescales are set to median values.)

In [ ]:
SAVE_DIR_DYN = 'data/simple/mlv_c/perturbed_ppln/c_baker_dyn_taufixed'
SAVE_PFX_DYN = 'mlv_c_baker_dyn_taufixed'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
tau_r_med = np.median(df_dyn_param['TAU_R'])
tau_a_med = np.median(df_dyn_param['TAU_A'])
print('Median TAU_R:', tau_r_med)
print('Median TAU_A:', tau_a_med)
df_dyn_param['TAU_R'] = tau_r_med
df_dyn_param['TAU_A'] = tau_a_med
df_dyn_param

In [ ]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

# Loop over value of homogeneous timescales

No adaptation -- homogeneous response timescale

In [ ]:
TAU_RS = [.1, .5, 1, 2, 3, 5, 7, 10, 15, 20, 30]
for tau_r in TAU_RS:
    print(f'Fixed TAU_R = {tau_r} s')
    SAVE_DIR_DYN = f'data/simple/mlv_c/perturbed_ppln/c_baker_dyn_taufixed_{tau_r}'
    SAVE_PFX_DYN = f'mlv_c_baker_dyn_taufixed_{tau_r}'

    if not os.path.exists(SAVE_DIR_DYN):
        os.makedirs(SAVE_DIR_DYN)

    # neural response fits from Baker data
    df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
    df_dyn_param['TAU_R'] = tau_r
    df_dyn_param['TAU_A'] = np.inf

    expt_ids = df_dyn_param['EXPT_ID']

    # get parameters
    tau_rs = np.array(df_dyn_param['TAU_R'])
    tau_as = np.array(df_dyn_param['TAU_A'])
    x_ss = np.array(df_dyn_param['X_S'])
    x_ps = np.array(df_dyn_param['X_P'])

    # compute surrogate neural responses for each trial
    for ctr, df_tr in enumerate(dfs_tr):
        df_tr_nrl_dyn = df_tr.copy()
        sys.stdout.write('.')
        # loop over neurons
        for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
            # compute surrogate neural response
            i_s = np.array(df_tr['S']).astype(float)
            i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)

            r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)

            df_tr_nrl_dyn[f'R_{expt_id}'] = r

        np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

# Dynamical model with non-selective neurons
(All responses respond equally to sine and pulse)

In [ ]:
SAVE_DIR_DYN = 'data/simple/mlv_c/perturbed_ppln/c_baker_dyn_nonselect'
SAVE_PFX_DYN = 'mlv_c_baker_dyn_nonselect'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
x_sp_mean = np.mean(np.array(df_dyn_param[['X_S', 'X_P']]), axis=1)
df_dyn_param['X_S'] = x_sp_mean
df_dyn_param['X_P'] = x_sp_mean
df_dyn_param

In [ ]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

# Dynamical model with purely selective neurons
(Every neuron responds only to its preferred song mode.)

In [ ]:
SAVE_DIR_DYN = 'data/simple/mlv_c/perturbed_ppln/c_baker_dyn_pureselect'
SAVE_PFX_DYN = 'mlv_c_baker_dyn_pureselect'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
x_s_alt = []
x_p_alt = []

for irow, row in df_dyn_param.iterrows():
    if np.abs(row['X_S']) > np.abs(row['X_P']):
        x_s_alt.append(row['X_S'])
        x_p_alt.append(0)
    else:
        x_s_alt.append(0)
        x_p_alt.append(row['X_P'])
        
df_dyn_param['X_S'] = x_s_alt
df_dyn_param['X_P'] = x_p_alt
df_dyn_param

In [ ]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

# Dynamical model with shuffled parameters
(Marginal param distributions across neurons are kept but correlations are broken.)

In [ ]:
SAVE_DIR_DYN = 'data/simple/mlv_c/perturbed_ppln/c_baker_dyn_paramshuffle'
SAVE_PFX_DYN = 'mlv_c_baker_dyn_paramshuffle'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

np.random.seed(0)
# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['TAU_R'] = np.array(df_dyn_param['TAU_R'])[np.random.permutation(len(df_dyn_param))]
df_dyn_param['TAU_A'] = np.array(df_dyn_param['TAU_A'])[np.random.permutation(len(df_dyn_param))]
df_dyn_param['X_S'] = np.array(df_dyn_param['X_S'])[np.random.permutation(len(df_dyn_param))]
df_dyn_param['X_P'] = np.array(df_dyn_param['X_P'])[np.random.permutation(len(df_dyn_param))]
df_dyn_param

In [ ]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

# Dynamical model with shuffled parameters downsampled
(Marginal param distributions across neurons are kept but correlations are broken.)

In [ ]:
NR = [150, 100, 50, 25, 10]

for nr in NR:
    print('NR =', nr)
    SAVE_DIR_DYN = f'data/simple/mlv_c/perturbed_ppln/c_baker_dyn_paramshuffle_nr_{nr}'
    SAVE_PFX_DYN = f'mlv_c_baker_dyn_paramshuffle_nr_{nr}'

    if not os.path.exists(SAVE_DIR_DYN):
        os.makedirs(SAVE_DIR_DYN)

    np.random.seed(0)
    # neural response fits from Baker data
    df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
    df_dyn_param['TAU_R'] = np.array(df_dyn_param['TAU_R'])[np.random.permutation(len(df_dyn_param))]
    df_dyn_param['TAU_A'] = np.array(df_dyn_param['TAU_A'])[np.random.permutation(len(df_dyn_param))]
    df_dyn_param['X_S'] = np.array(df_dyn_param['X_S'])[np.random.permutation(len(df_dyn_param))]
    df_dyn_param['X_P'] = np.array(df_dyn_param['X_P'])[np.random.permutation(len(df_dyn_param))]

    expt_ids = np.arange(nr, dtype=int)

    # get parameters
    tau_rs = np.array(df_dyn_param['TAU_R'])[:nr]
    tau_as = np.array(df_dyn_param['TAU_A'])[:nr]
    x_ss = np.array(df_dyn_param['X_S'])[:nr]
    x_ps = np.array(df_dyn_param['X_P'])[:nr]

    # compute surrogate neural responses for each trial
    for ctr, df_tr in enumerate(dfs_tr):
        df_tr_nrl_dyn = df_tr.copy()
        sys.stdout.write('.')
        # loop over neurons
        for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
            # compute surrogate neural response
            i_s = np.array(df_tr['S']).astype(float)
            i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)

            r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)

            df_tr_nrl_dyn[f'R_{expt_id}'] = r

        np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

# Dynamical model with halved response timescalse

In [ ]:
SAVE_DIR_DYN = 'data/simple/mlv_c/perturbed_ppln/c_baker_dyn_taurhalf'
SAVE_PFX_DYN = 'mlv_c_baker_dyn_taurhalf'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['TAU_R'] = df_dyn_param['TAU_R']/2
df_dyn_param

In [ ]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

# Dynamical model with doubled response timescales

In [ ]:
SAVE_DIR_DYN = 'data/simple/mlv_c/perturbed_ppln/c_baker_dyn_taurdbl'
SAVE_PFX_DYN = 'mlv_c_baker_dyn_taurdbl'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['TAU_R'] = df_dyn_param['TAU_R']*2
df_dyn_param

In [ ]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))